In [23]:
# Import statements
import pandas as pd
import numpy as np
import sklearn.model_selection
import sklearn.compose # column transformers
import sklearn.preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [24]:
# read in data
data = pd.read_csv("dataset_diabetes/diabetic_data.csv").drop_duplicates(["patient_nbr"])

In [25]:
X, y = data.drop("readmitted", axis=1), data["readmitted"]

In [26]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, \
                                                    test_size = 0.20, train_size = 0.80, shuffle = True)

Weight is missing from 97% of the observations in the dataset, while payer and admitting physician specialty are missing from 52% and 53% respectively. We cut each of these three variables.

In [27]:
Xtrain = (Xtrain[Xtrain.columns[~Xtrain.columns.str.contains("_id")]] #drop all id columns
            .drop("patient_nbr", axis=1) # drop other id column
            .drop(["weight", "payer_code", "medical_specialty"], axis=1) # drop incomplete columns
            .replace({"gender": {"Female": 1, "Male": 0, "Unknown/Invalid": 2}, 
                      "race": {"AfricanAmerican": 0, "Hispanic": 1, "Asian": 2, "Caucasian": 3, "Other": 4},
                      "age": {"[0-10)": 0, "[10-20)": 1, "[20-30)": 2, "[30-40)": 3, 
                              "[40-50)": 4, "[50-60)": 5, "[60-70)": 6, "[70-80)": 7, 
                              "[80-90)": 8, "[90-100)": 9},
                      "max_glu_serum": {"None": 0, "Norm": 1, ">200": 2, ">300": 3},
                      "A1Cresult": {"None": 0, "Norm": 1, ">7": 2, ">8": 3}
                      }) #recode categorical as factors
            .replace({'?': None, "No": 0, "Yes": 1, "Ch": 1}) # null entries are recorded as '?' in the data
            .replace({"Steady": 1, "Down": 2, "Up": 3})
            #.dropna(axis=0) # drop null rows for now
)
ytrain = (ytrain
          .replace({"NO": 0, ">30": 1, "<30": 1})
)

In [28]:
# let's try to train a simple model just based off the numeric features
dfx = (Xtrain#[["num_lab_procedures", "num_medications", "time_in_hospital", "num_procedures"]]
      .drop(["diag_1", "diag_2", "diag_3"], axis=1)
      .apply(pd.to_numeric, axis=1)
)
dfy = pd.to_numeric(ytrain)
dfx_train, dfx_test, dfy_train, dfy_test = train_test_split(dfx, dfy, test_size = 0.20, train_size = 0.80, shuffle = True)

In [16]:
df = pd.concat([dfx_train, dfy_train], axis=1)
df = (df
      .replace([np.inf, -np.inf, np.NaN], np.nan) 
      .dropna(axis=0) #drop any rows we can't train on for now
      
)
print("We have %d rows left after dropping rows with NA in them." % df.shape[0])
dfx_train, dfy_train = df.drop("readmitted", axis=1), df["readmitted"]

We have 44537 rows left after dropping rows with NA in them.


In [ ]:
df = pd.concat([dfx_train, dfy_train], axis=1)
scaler = MinMaxScaler().fit(df)
df_scaled = scaler.transform(df)

In [30]:
df_scaled.to_csv("scaled_data.csv", index=False)

In [ ]:
def binary_converter():
    
binary_transformer = sklearn.preprocessing.FunctionTransformer(binary_converter)

In [ ]:
## full preprocessing pipeline from the training data
pipeline = Pipeline()

In [ ]:
## test it out on the cv data

In [ ]:
## and export it to a file for use on the test data